## Assignment 1 
### Anubhav a1812913

### 1. Reading datasets

In [2]:
#import Libraries
import string
# nltk.download('stopwords')  
from nltk.corpus import stopwords #Importing Stop Words
#Tokenize the words
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize #to TOkenize the words
#import lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from google.colab import drive
drive.mount('/content/drive')
# with open("drive/My Drive/NLP/Ass1/Data/reviews_train.txt", encoding='latin-1') as f:
#     contents = f.readlines()


#reading data
train_text = []
with open('drive/My Drive/NLP/Ass1/Data/reviews_train.txt', encoding='latin-1') as f:
    train_text = f.readlines()
train_text

# #to Tokenize the words
# token_words_train = [word_tokenize(text) for text in train_text]
# token_words_train

# # First convert to lowercase the train data
# train_text_lower = [word.lower() for word in token_words_train ]

# # Second correct the punctuation of training data
# filter_train_text = [i.translate(i.maketrans('', '', string.punctuation)) for i in train_text_lower]




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['__label__1 Almost Completely Useless: It is impossible to keep the Ethernet cord plugged in to this device. It just slides out. I have had to use tape, zip ties, and a Popsicle stick to keep it plugged in. Terrible product.\n',
 '__label__1 Not enough forskolin: Ordered this not reading that it only has a guarantee of 3.85 mg of forskolin. That means you would have to take almost 13 pills a day to get the scientific recommended amount you need to take each day which is 50 mg of active forskolin. Sorry but not impressed. Buy another product that actually has more than 10 mg per serving which is a much greater value.\n',
 '__label__1 Item not received: The item was never received and after 3 e-mails, I have not heard from the company about refunding my money or sending the product.\n',
 '__label__2 Great episode: If you only watch a handful of Next Gen episodes, The Best of Both Worlds Part 1 and 2 are the way to go.\n',
 "__label__2 ALL TIME FAVORTITE BOOK!! MUST READ!!: This is the B

In [ ]:
#import lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Extract lalbels
token_words_train_labels = [i[0] for i in filter_train_textUy]
# token_words_train_labels

#Remove labels form Training data
train_data = [i[1:-1] for i in token_words_train]
train_data

#Removing Stop words
stop_words = set(stopwords.words('english'))
Fiter_stop_train =[]
for line in train_data:
    fltr_line = [word for word in line if word not in stop_words]
    Fiter_stop_train.append(list(fltr_line))
# Fiter_stop_train

#Lemmatizing sentences 
lemmatized_words_train = []
for line in Fiter_stop_train:
    lemma_line = [lemmatizer.lemmatize(word) for word in line]
    lemmatized_words_train.append(list(lemma_line))
lemmatized_words_train

[['almost',
  'completely',
  'useless',
  'impossible',
  'keep',
  'ethernet',
  'cord',
  'plugged',
  'device',
  'slide',
  'use',
  'tape',
  'zip',
  'tie',
  'popsicle',
  'stick',
  'keep',
  'plugged',
  'terrible'],
 ['enough',
  'forskolin',
  'ordered',
  'reading',
  'guarantee',
  '385',
  'mg',
  'forskolin',
  'mean',
  'would',
  'take',
  'almost',
  '13',
  'pill',
  'day',
  'get',
  'scientific',
  'recommended',
  'amount',
  'need',
  'take',
  'day',
  '50',
  'mg',
  'active',
  'forskolin',
  'sorry',
  'impressed',
  'buy',
  'another',
  'product',
  'actually',
  '10',
  'mg',
  'per',
  'serving',
  'much',
  'greater'],
 ['item',
  'received',
  'item',
  'never',
  'received',
  '3',
  'email',
  'heard',
  'company',
  'refunding',
  'money',
  'sending'],
 ['great',
  'episode',
  'watch',
  'handful',
  'next',
  'gen',
  'episode',
  'best',
  'world',
  'part',
  '1',
  '2',
  'way'],
 ['time',
  'favortite',
  'book',
  'must',
  'read',
  'best',

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# load the dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# limit to the first 1000 articles
# num_rows = 10000
num_rows = len(newsgroups.data) # full size

# split the dataset into training and test sets
train_data = newsgroups.data[:int(num_rows*0.8)]
train_target = newsgroups.target[:int(num_rows*0.8)]
test_data = newsgroups.data[int(num_rows*0.8):]
test_target = newsgroups.target[int(num_rows*0.8):]

In [ ]:
import numpy as np
np.unique(train_target)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

### 2. Pre-processing

### 3. Dataset analysis

### 4. Sentiment classification using Machine Learning

### 5. Sentiment classification using VADER sentiment lexicon

### 6. (optional challenge 3 points) Combine VADER with the classifier

### 7. References